# LlamaIndex와 AgentCore Memory - 학술 연구 어시스턴트 (장기 메모리)

## 소개

이 노트북은 Amazon Bedrock AgentCore Memory 기능을 LlamaIndex와 통합하여 **장기 메모리** 지속성을 갖춘 학술 연구 어시스턴트를 만드는 방법을 보여줍니다. 여러 연구 세션에 걸쳐 메모리가 유지되어 어시스턴트가 수주 및 수개월의 연구 작업 동안 누적 지식을 구축할 수 있습니다.

## 아키텍처 개요

![LlamaIndex AgentCore Long-Term Memory Architecture](LlamaIndex-AgentCore-LTM-Arch.png)

## 튜토리얼 세부사항

**튜토리얼 세부사항:**
- **튜토리얼 유형**: 장기 세션 간 메모리
- **Agent 사용 사례**: 학술 연구 어시스턴트
- **Agentic Framework**: LlamaIndex
- **LLM model**: Anthropic Claude 3.7 Sonnet
- **튜토리얼 구성요소**: AgentCore 장기 메모리, LlamaIndex Agent, 연구 도구
- **예제 복잡도**: 고급

## 비즈니스 가치

**기업 연구 인텔리전스**: 기관 지식을 축적하고, 연구 진화를 추적하며, 프로젝트와 기간에 걸쳐 포괄적인 학술 컨텍스트를 유지하는 지속적인 AI 메모리로 연구 워크플로우를 혁신하세요.

**주요 전문적 이점:**
- **연구 연속성**: 연구 단계와 팀 구성원 간의 원활한 지식 전달
- **기관 메모리**: 중요한 연구 통찰력, 방법론 및 결과를 영구적으로 보존
- **프로젝트 간 인텔리전스**: 여러 연구 이니셔티브에 걸친 패턴과 연결 식별
- **연구비 제안 우수성**: 설득력 있는 자금 신청을 위한 과거 연구 데이터 활용
- **학술 협업**: 다년간의 협력 연구 프로젝트를 위한 상세한 컨텍스트 유지
- **출판 전략**: 전략적 출판 계획을 위한 연구 주제 및 인용 네트워크 추적

## 장기 메모리 구성

**기술 설정**: 이 튜토리얼은 12개월 보존을 위한 Semantic Strategy와 함께 AgentCore Memory를 사용합니다:
- **메모리 유형**: 자동 통찰력 추출을 갖춘 Semantic strategy
- **보존 기간**: 연구 연속성을 위한 365일 이벤트 만료
- **세션 간**: 동일한 actor_id + memory_id, 연구 기간별로 다른 session_id
- **검색 기능**: 연구 기록 전반에 걸친 의미론적 검색을 위한 내장 메모리 검색 도구

## 기술 개요

**주요 장기 메모리 구성요소:**
1. **Semantic Strategy 구성**: 365일 보존과 함께 자동 통찰력 추출을 위한 SemanticStrategy 사용
2. **세션 간 지속성**: 동일한 actor_id + memory_id, 기간별로 다른 session_id로 지식 연속성 활성화
3. **커스텀 메모리 검색 도구**: AgentCore의 네이티브 search_long_term_memories()를 LlamaIndex FunctionTool로 래핑
4. **의미론적 처리 파이프라인**: 대화 이벤트 → 의미론적 메모리 변환을 위한 90초 대기
5. **동적 세션 관리**: 유연한 세션 처리를 위해 memory.context.session_id 사용

**학습 내용:**

- 여러 연구 세션에 걸쳐 지속적인 AgentCore Memory 생성
- 시간이 지남에 따라 누적 연구 지식 구축
- 연구 기록 전반에 걸친 의미론적 검색 구현
- 연구 진화 및 전문성 개발 추적
- 세션 간 메모리 지속성 및 검색 테스트

## 사전 요구사항

- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory 권한이 있는 AWS IAM 역할:
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock models에 대한 액세스

## 단계 1: 종속성 설치 및 설정

In [ ]:
# Install necessary libraries including semantic strategy toolkit
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3 bedrock-agentcore-starter-toolkit

In [ ]:
# Import required components
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore_starter_toolkit.operations.memory.manager import MemoryManager
from bedrock_agentcore.memory.session import MemorySessionManager
from bedrock_agentcore_starter_toolkit.operations.memory.models.strategies.semantic import SemanticStrategy
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

print("✅ All dependencies imported successfully!")

## 단계 2: AgentCore Memory 구성

장기 연구 지식을 위한 AgentCore Memory 리소스를 생성하거나 가져옵니다:

In [ ]:
# Create AgentCore Memory with Semantic Strategy for long-term persistence
region = os.getenv('AWS_REGION', 'us-east-1')
memory_manager = MemoryManager(region_name=region)

try:
    # SemanticStrategy: 대화에서 자동으로 통찰력을 추출하여 장기 메모리로 저장
    memory = memory_manager.get_or_create_memory(
        name=f'AcademicResearchSemantic_{int(datetime.now().timestamp())}',
        strategies=[SemanticStrategy(name="researchLongTermMemory")],
        event_expiry_days=365  # 365일 동안 이벤트 보존
    )
    memory_id = memory.get('id')
    print(f"✅ Created Semantic Memory: {memory_id}")
    print(f"   Status: {memory.get('status')}")
    print(f"   Strategies: {[s.get('name') if isinstance(s, dict) else str(s) for s in memory.get('strategies', [])]}")
    
    # 메모리가 ACTIVE 상태가 될 때까지 대기 (사용 가능 상태)
    if memory.get('status') != 'ACTIVE':
        print(f"\n⏳ Waiting for memory to become ACTIVE (currently {memory.get('status')})...")
        import time
        max_wait = 300  # 최대 5분 대기
        waited = 0
        while waited < max_wait:
            time.sleep(10)
            waited += 10
            current_memory = memory_manager.get_memory(memory_id)
            status = current_memory.get('status')
            print(f"   [{waited}s] Status: {status}")
            if status == 'ACTIVE':
                print(f"✅ Memory is now ACTIVE! (took {waited} seconds)")
                break
        else:  # while 루프가 break 없이 종료된 경우
            print(f"⚠️  Memory still not ACTIVE after {max_wait}s. Proceeding anyway...")
    
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # 기존 memory ID로 교체

## 단계 3: 연구 도구 구현

학술 연구 작업을 위한 전문 도구를 정의합니다:

In [ ]:
def save_paper_summary(title: str, authors: str, key_findings: str) -> str:
    """Save a research paper summary with title, authors, and key findings"""
    print(f"📄 Saved paper: {title} by {authors}")
    return f"Successfully saved paper summary for '{title}'"

def track_research_topic(topic: str, status: str) -> str:
    """Track research topic progress with current status"""
    print(f"🔬 Tracking research topic: {topic} (Status: {status})")
    return f"Now tracking research topic: {topic} with status {status}"

def save_research_finding(finding: str, confidence: str) -> str:
    """Save a research finding with confidence level"""
    print(f"💡 Research finding saved with {confidence} confidence")
    return f"Saved research finding with {confidence} confidence level"

def update_research_status(topic: str, new_status: str, notes: str) -> str:
    """Update research topic status with notes"""
    print(f"📊 Updated {topic} status to: {new_status}")
    return f"Updated research status for {topic}"

def log_research_milestone(period: str, milestone: str, details: str) -> str:
    """Log a research milestone with period and detailed progress"""
    print(f"🎯 {period} milestone: {milestone}")
    return f"Logged milestone for {period}: {milestone} - {details}"

def track_research_metrics(metric_type: str, value: str, source: str, period: str) -> str:
    """Track specific research metrics with source and timeline"""
    print(f"📊 {period}: {metric_type} = {value} (from {source})")
    return f"Tracked {metric_type}: {value} from {source} in {period}"

def save_research_insight(insight: str, period: str, connections: str) -> str:
    """Save research insights with connections to previous work"""
    print(f"💡 {period} insight: {insight[:50]}...")
    return f"Saved {period} insight with connections: {connections}"

# Create tool objects for the agent
research_tools = [
    FunctionTool.from_defaults(fn=save_paper_summary),
    FunctionTool.from_defaults(fn=track_research_topic),
    FunctionTool.from_defaults(fn=save_research_finding),
    FunctionTool.from_defaults(fn=update_research_status),
    FunctionTool.from_defaults(fn=log_research_milestone),
    FunctionTool.from_defaults(fn=track_research_metrics),
    FunctionTool.from_defaults(fn=save_research_insight)
]

print("✅ Research tools created!")

## 단계 3b: 메모리 검색 도구 추가

Agent가 장기 메모리를 검색할 수 있도록 하는 도구를 생성합니다:

In [ ]:
def create_memory_retrieval_tool(memory_id: str, actor_id: str, region: str):
    """Agent가 장기 메모리를 검색할 수 있는 도구 생성"""
    
    def search_long_term_memory(query: str) -> str:
        """Search long-term memory for relevant research information.
        
        Use this tool when you need to recall:
        - Previous research papers and findings
        - Research topics and their status
        - Metrics and insights from past work
        - Research milestones and progress
        
        Args:
            query: Search query describing what information you need
        
        Returns:
            Relevant information from long-term memory
        """
        try:
            from bedrock_agentcore.memory.session import MemorySessionManager
            
            session_manager = MemorySessionManager(
                memory_id=memory_id,
                region_name=region
            )
            
            # semantic strategy namespace에서 장기 메모리 검색
            results = session_manager.search_long_term_memories(
                query=query,
                namespace_prefix="/strategies/",  # semantic strategy의 메모리 위치
                top_k=5,  # 상위 5개 결과
                max_results=10  # 최대 10개 결과
            )
            
            if not results:
                return "No relevant information found in long-term memory. This might be new information or the memory extraction may still be processing."
            
            # Agent가 이해할 수 있도록 결과 포맷팅
            output = "📚 Retrieved from long-term memory:\\n\\n"
            for i, result in enumerate(results, 1):
                # MemoryRecord 객체에서 content 속성 추출
                content = getattr(result, 'content', str(result))
                # 너무 긴 내용은 잘라내기
                if len(content) > 300:
                    content = content[:300] + "..."
                output += f"{i}. {content}\\n\\n"
            
            return output
            
        except Exception as e:
            return f"⚠️ Error searching memory: {str(e)}. Proceeding without historical context."
    
    return FunctionTool.from_defaults(fn=search_long_term_memory)

# 메모리 검색 도구 생성
memory_search_tool = create_memory_retrieval_tool(memory_id, "academic-researcher", region)

# 기존 도구 목록에 메모리 검색 추가
research_tools_with_memory = research_tools + [memory_search_tool]

print(f"✅ Memory retrieval tool created! Total tools: {len(research_tools_with_memory)}")
print("   Using namespace: /strategies/ (for semantic strategy compatibility)")

## 단계 3c: 메모리 구성 확인

Semantic strategy가 올바르게 구성되었는지 확인합니다:

In [ ]:
# Check memory configuration
memory_info = memory_manager.get_memory(memory_id)
print(f"Strategies: {memory_info.get('strategies')}")
print(f"Status: {memory_info.get('status')}")
print(f"Name: {memory_info.get('name')}")

# Show strategy details
strategies = memory_info.get('strategies', [])
for strategy in strategies:
    print(f"\nStrategy Details:")
    print(f"  Name: {strategy.get('name')}")
    print(f"  Type: {strategy.get('type')}")
    print(f"  Status: {strategy.get('status')}")
    print(f"  ID: {strategy.get('strategyId')}")

## 단계 4: 다중 세션 Agent 구현

다양한 연구 세션을 시뮬레이션하는 헬퍼 함수를 생성합니다:

In [ ]:
# Configuration for LONG-TERM memory (cross-session)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
RESEARCHER_ID = "academic-researcher"  # 모든 세션에서 동일한 연구자 ID

def create_research_session(session_name: str):
    """장기 메모리 지속성을 갖춘 새로운 연구 세션 생성"""
    context = AgentCoreMemoryContext(
        actor_id=RESEARCHER_ID,         # 동일한 연구자 (세션 간 메모리 공유)
        memory_id=memory_id,         # 동일한 메모리 저장소 (장기 메모리 활성화)
        session_id=f"research-{session_name}", # 기간별로 다른 세션 ID
        namespace="/academic-research/"
    )
    
    memory = AgentCoreMemory(context=context)
    llm = BedrockConverse(model=MODEL_ID)
    agent = FunctionAgent(
        tools=research_tools_with_memory,  # 메모리 검색 기능이 포함된 도구 사용
        llm=llm, 
        verbose=True,  # 메모리 검색 시점을 확인하기 위해 verbose 활성화
        system_prompt="""You are a senior research assistant with access to long-term memory.
        
CRITICAL: When asked about previous research, papers, findings, or historical information, 
you MUST use the search_long_term_memory tool FIRST before responding.

For example:
- "What research am I working on?" → Use search_long_term_memory("research topics")
- "What papers have I reviewed?" → Use search_long_term_memory("papers authors")
- "What findings do I have?" → Use search_long_term_memory("research findings")

Always provide conclusive, complete responses without asking follow-up questions.\n
Execute all requested actions immediately and completely. Provide detailed, professional responses."""
    )
    
    return agent, memory

print("✅ Multi-session Academic Research Assistant setup complete!")


## 단계 5: 1주차 연구 세션 - 기초 구축

첫 번째 연구 세션을 시작하고 기초 지식을 확립합니다:

In [ ]:
# === WEEK 1 RESEARCH SESSION ===
print("🗓️ === WEEK 1: FOUNDATION RESEARCH ===")

agent_week1, memory_week1 = create_research_session("week1")

# Establish research foundation
response = await agent_week1.run(
    "I'm Dr. Sarah Smith from MIT starting comprehensive research on 'Machine Learning in Healthcare Applications'. "
    "Track this with status 'Literature Review'. My goal is to publish a systematic review by year-end.",
    memory=memory_week1
)

print("🎯 Week 1 Foundation:")
print(response)

In [ ]:
# Add foundational papers with detailed metrics
response = await agent_week1.run(
    "Save paper: 'Deep Learning for Medical Image Analysis' by Zhang et al (2023). "
    "Key findings: CNNs achieve 95.2% accuracy in chest X-ray diagnosis, 12% improvement over radiologists, "
    "trained on 100,000 images, 0.03 false positive rate.",
    memory=memory_week1
)
print("📄 Week 1 Paper 1:", response)

response = await agent_week1.run(
    "Save paper: 'Transformers in Medical NLP' by Johnson et al (2023). "
    "Key findings: BERT achieves 89.1% F1-score in clinical note classification, "
    "struggles with rare diseases (<70% accuracy), excels at symptom extraction (94% precision).",
    memory=memory_week1
)
print("📄 Week 1 Paper 2:", response)
# 정확도 메트릭을 명시적으로 추적 (장기 메모리에 저장)
await agent_week1.run(
    "Track research metrics: metric_type 'CNN Accuracy', value '95.2%', source 'Zhang et al 2023', period 'Week 1'.",
    memory=memory_week1
)
await agent_week1.run(
    "Track research metrics: metric_type 'Radiologist Improvement', value '12%', source 'Zhang et al 2023', period 'Week 1'.",
    memory=memory_week1
)


In [ ]:
# Allow time for semantic memory processing
import asyncio
print("\n⏳ Waiting for semantic memory extraction and indexing...")
print("   (AgentCore processes conversational events in the background)")
await asyncio.sleep(90)  # semantic strategy가 대화를 처리하고 메모리로 변환하는 시간
print("✅ Memory processing complete - memories should now be searchable")


## 단계 6: 2주차 연구 세션 - 세션 간 메모리 테스트

장기 메모리 검색을 테스트하고 새로운 연구를 추가합니다:

In [ ]:
# === WEEK 2 RESEARCH SESSION ===
print("\n🗓️ === WEEK 2: EXPANSION (NEW SESSION) ===")

agent_week2, memory_week2 = create_research_session("week2")  # 새로운 세션 생성

# 세션 간 메모리 회상 테스트 (Week 1의 정보를 Week 2에서 검색)
response = await agent_week2.run(
    "What research am I working on? What specific accuracy metrics have I found so far? Who are the key authors?",
    memory=memory_week2
)

print("🧠 Week 2 Memory Test:")
print(response)
print("\n✅ Expected: ML in Healthcare, Zhang 95.2%, Johnson 89.1% F1-score")

In [ ]:
# Add new research building on previous knowledge
response = await agent_week2.run(
    "Save paper: 'Federated Learning in Healthcare' by Brown et al (2023). "
    "Key findings: Privacy-preserving ML enables multi-hospital collaboration, 87.3% accuracy across 15 hospitals, "
    "23% improvement in rare disease detection when hospitals collaborate.",
    memory=memory_week2
)
print("📄 Week 2 New Paper:", response)

# 세션 간 비교 분석 테스트 (Week 1과 Week 2의 정보를 종합)
response = await agent_week2.run(
    "Compare the accuracy results: Zhang's CNNs vs Johnson's BERT vs Brown's federated learning. "
    "Which performs best and in what contexts?",
    memory=memory_week2
)
print("📊 Week 2 Comparative Analysis:")
print(response)
print("\n✅ Expected: Zhang 95.2% (imaging), Johnson 89.1% (NLP), Brown 87.3% (federated)")

## 단계 7: 3주차 연구 세션 - 분석 단계

연구를 진행하고 세부적인 세션 간 회상을 테스트합니다:

In [ ]:
# === WEEK 3 RESEARCH SESSION ===
print("\n🗓️ === WEEK 3: ANALYSIS PHASE ===")

agent_week3, memory_week3 = create_research_session("week3")

# Update research status
response = await agent_week3.run(
    "Update my 'Machine Learning in Healthcare Applications' research status to 'Analysis Phase' "
    "with notes: 'Reviewed 3 key papers, identified performance patterns: imaging>NLP>federated learning'.",
    memory=memory_week3
)
print("📊 Week 3 Status Update:", response)

# 세부적인 세션 간 회상 테스트 (특정 주장에 대한 증거 검색)
response = await agent_week3.run(
    "What evidence do I have for the claim that imaging tasks show highest ML performance in healthcare? "
    "Include specific numbers and authors.",
    memory=memory_week3
)
print("🔍 Week 3 Evidence Query:")
print(response)
print("\n✅ Expected: Zhang et al CNNs 95.2% vs Johnson BERT 89.1% vs Brown federated 87.3%")

## 단계 8: 1개월차 연구 세션 - 종합 단계

포괄적인 지식 종합 및 연구 통합을 테스트합니다:

In [ ]:
# === MONTH 1 RESEARCH SESSION ===
print("\n🗓️ === MONTH 1: SYNTHESIS PHASE ===")

agent_month1, memory_month1 = create_research_session("month1")

# Update research status to synthesis phase
response = await agent_month1.run(
    "Update my 'Machine Learning in Healthcare Applications' research status to 'Synthesis Phase' "
    "with notes: 'Completed 3-week literature review, ready to synthesize findings into coherent framework'.",
    memory=memory_month1
)
print("📊 Month 1 Status Update:", response)

# 모든 주차의 정보를 종합하는 포괄적 테스트
response = await agent_month1.run(
    "Based on all my research so far, what is the overall performance ranking of ML approaches in healthcare? "
    "Include all specific metrics and create a comprehensive comparison.",
    memory=memory_month1
)
print("🔍 Month 1 Comprehensive Synthesis:")
print(response)
print("\n✅ Expected: Ranking with Zhang 95.2% > Johnson 89.1% > Brown 87.3%, domain analysis")

## 단계 9: 2개월차 연구 세션 - 작성 단계

포괄적인 회상 및 의미론적 검색 기능을 테스트합니다:

In [ ]:
# === MONTH 2 RESEARCH SESSION ===
print("\n🗓️ === MONTH 2: WRITING PHASE ===")

agent_month2, memory_month2 = create_research_session("month2")

# 논문 작성을 위한 포괄적 회상 테스트
response = await agent_month2.run(
    "I'm writing my systematic review paper. What are ALL the papers I've reviewed with their exact accuracy metrics? "
    "I need this for my results table.",
    memory=memory_month2
)
print("📝 Month 2 Comprehensive Recall:")
print(response)
print("\n✅ Expected: Zhang 95.2%, Johnson 89.1%, Brown 87.3% with full details")

In [ ]:
# 연구 기록 전반에 걸친 의미론적 검색 테스트
response = await agent_month2.run(
    "What do I know about rare disease detection in my research? Which papers and what specific results?",
    memory=memory_month2
)
print("🔍 Month 2 Semantic Search:")
print(response)
print("\n✅ Expected: Johnson <70% for rare diseases, Brown 23% improvement with collaboration")

## 단계 10: 3개월차 연구 세션 - 연구비 제안 시나리오

축적된 지식의 실용적 적용을 테스트합니다:

In [ ]:
# === MONTH 3 RESEARCH SESSION ===
print("\n🗓️ === MONTH 3: GRANT PROPOSAL ===")

agent_month3, memory_month3 = create_research_session("month3")

# 연구비 제안서를 위한 증거 수집
response = await agent_month3.run(
    "I'm writing an NIH grant proposal for $2M funding. What evidence can I cite about ML effectiveness in healthcare? "
    "I need specific numbers, authors, years, and sample sizes.",
    memory=memory_month3
)
print("💰 Month 3 Grant Evidence:")
print(response)
print("\n✅ Expected: Comprehensive citation with Zhang 95.2% (100K images), Johnson 89.1%, Brown 87.3% (15 hospitals)")

In [ ]:
# 연구 진화 추적 테스트 (시간에 따른 연구 발전 과정)
response = await agent_month3.run(
    "Provide a detailed timeline of my research evolution from Week 1 to now. What specific milestones, "
    "metrics, and insights did I achieve each period? How did my research questions evolve?",
    memory=memory_month3
)
print("📈 Month 3 Research Evolution:")
print(response)
print("\n✅ Expected: Week-by-week progression with specific milestones, metrics (95.2%, 89.1%, 87.3%), and insights")

## 단계 11: 최종 포트폴리오 평가

장기 메모리 기능의 포괄적인 테스트:

In [ ]:
# 최종 포괄적 포트폴리오 쿼리
response = await agent_month3.run(
    "Provide my complete research portfolio: all topics I'm working on, all papers with metrics, "
    "all findings, current status of each project, and how they interconnect.",
    memory=memory_month3
)
print("📋 Complete Research Portfolio:")
print(response)
print("\n✅ Expected: Full research history with all metrics, connections between ML healthcare topics")

## 🧪 자동화된 테스트 검증
메모리 통합이 올바르게 작동하는지 검증하려면 다음 셀을 실행하세요:

In [ ]:
# Define validation functions inline
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """세션 내에서 이전 정보를 회상할 수 있는지 확인"""
        # 실질적인 응답 확인 (단순히 "모르겠습니다"가 아닌)
        has_content = len(response) > 50
        # 메모리 지표 확인
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """세션 내에서 컨텍스트를 유지하는지 확인"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """현재 쿼리를 이전 컨텍스트와 연결할 수 있는지 확인"""
        # 연결 언어 확인
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# Run all validation tests
test_results = {}

# Test 1: 메모리 회상 - agent가 논의된 내용을 기억하는가?
response1 = await agent_month3.run("What have we discussed so far in this session?", memory=memory_month3)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"Response 1 length: {len(str(response1))} chars")

# Test 2: 세션 메모리 - agent가 컨텍스트를 유지하는가?
response2 = await agent_month3.run("What did we talk about earlier?", memory=memory_month3)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"Response 2 length: {len(str(response2))} chars")

# Test 3: 교차 참조 능력 - 이전 컨텍스트와 연결할 수 있는가?
response3 = await agent_month3.run("How does this relate to what we discussed before?", memory=memory_month3)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"Response 3 length: {len(str(response3))} chars")

# Display results
validator.run_validation_summary(test_results)

## 요약

이 노트북에서는 다음을 시연했습니다:

✅ **장기 메모리 통합**: 세션 간 지속성을 위해 LlamaIndex와 함께 AgentCore Memory 사용

✅ **누적 지식 구축**: 수주 및 수개월에 걸쳐 연구 지식 축적

✅ **의미론적 검색**: 어시스턴트가 세션 간에 개념을 기반으로 관련 정보를 찾을 수 있음

✅ **연구 진화 추적**: 문헌 검토에서 분석, 작성까지의 자연스러운 진행

✅ **세션 간 종합**: 여러 연구 세션에 걸쳐 결과 및 통찰력 연결

✅ **실용적 적용**: 연구비 제안 지원 및 포괄적인 포트폴리오 관리

학술 연구 어시스턴트는 장기 메모리가 어시스턴트를 시간이 지남에 따라 더 똑똑해지는 지속적인 연구 동반자로 변환하여 완전한 연구 기록을 유지하고 확장된 연구 프로젝트에 걸쳐 정교한 지식 검색을 가능하게 하는 방법을 보여줍니다.

## 정리

이 노트북에서 사용된 리소스를 정리하기 위해 메모리를 삭제하겠습니다:

**참고**: 메모리를 영구적으로 삭제하려는 경우에만 실행하세요. memory_id 변수는 이 노트북의 앞부분에서 생성된 메모리의 ID를 포함해야 합니다.

In [ ]:
# Clean up AgentCore Memory resource
try:
    from bedrock_agentcore.memory import MemoryClient
    
    client = MemoryClient(region_name=region)
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
    
except NameError as e:
    print(f"⚠️  Variable not defined: {e}")
    print("Run the notebook from the beginning or set variables manually:")
    print("# memory_id = 'your-memory-id-here'")
    print("# region = 'us-east-1'")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")